# Convex Hulls

General information

## Table of Contents

1. Introduction
2. Imports
3. Algorithms
    * Naive Hull
    * Graham Scan
    * Gift Wrapping
    * Chan's Hull
4. Robustness
5. Experiments
6. Conclusion

## 2. Imports

In [ ]:
from enum import Enum, auto
import math


class Or(Enum):
    LEFT = auto()
    ON = auto()
    RIGHT = auto()


class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __repr__(self):
        return f"Point({self.x}, {self.y})"
    
    #def as_np(self):
    #    return np.array([self.x, self.y])
    
    def distance(self, other: 'Point') -> float:
        return math.sqrt((self.x - other.x)**2 + (self.y - other.y)**2)
    
    def orientation(self, p: 'Point', q: 'Point') -> Or:   # p != q ...
        val = (q.x - p.x) * (self.y - p.y) - (q.y - p.y) * (self.x - p.x)
        if val > 0.0:
            return Or.LEFT
        elif val == 0.0:
            return Or.ON
        else:
            return Or.RIGHT

In [ ]:
from ipycanvas import Canvas
canvas = Canvas(width=400, height=250)

canvas.fill_style = 'blue'
canvas.stroke_style = 'purple'
canvas.line_width = 5

## 3. Algorithms

### 3.1 Naive Hull
The first algorithm from the lecture

In [ ]:
def naive_hull(points: list[Point]) -> list[Point]:
    edges: dict[Point, Point] = {}
    for p in points:
        for q in filter(lambda q: q != p, points):
            valid = True
            for r in filter(lambda r: r != p and r != q, points):
                if r.orientation(p, q) is Or.RIGHT:
                    valid = False
                    break
            if valid and (p not in edges or p.distance(q) > p.distance(edges[p])):
                if p in edges and egdes[p] in edges:
                    del edges[edges[p]]
                edges[p] = q
    hull: list[Point] = []
    if edges:
        first_point, next_point = edges.popitem()
        hull.append(first_point)
        while next_point is not None and next_point != hull[0]:
            hull.append(next_point)
            next_point = edges.pop(next_point, None)
    return hull

In [ ]:
points = [Point(0,0), Point(100,50), Point(250,25), Point(100,100), Point(200,200), Point(300,0)]
hull = naive_hull(points)
print(hull)

In [ ]:
canvas.fill_polygon(list(map(lambda p: (p.x, 250 - p.y), hull)))
canvas.stroke_polygon(list(map(lambda p: (p.x, 250 - p.y), hull)))

canvas.fill_style = "orange"
canvas.fill_circles(list(map(lambda p: p.x, points)), list(map(lambda p: 250 - p.y, points)), [5,5,5,5,5,5])

canvas

should handle degeneracies (sure?), but it's not robust

### 3.2 Graham Scan
Better algorithm

In [ ]:
def graham_scan(points: list[Point]) -> list[Point]:    
    #if len(points) <= 2:
    #    return points[:]
    sorted_points = sorted(points, key = lambda p: (p.x, p.y))
    upper_hull = graham_half_scan(sorted_points)
    sorted_points.reverse()
    lower_hull = graham_half_scan(sorted_points)
    hull = upper_hull[:-1] + lower_hull[:-1]
    return hull

def graham_half_scan(points: list[Point]) -> list[Point]:
    if len(points) <= 2:
        return points[:]
    hull = points[:2]
    for p in points[2:]:
        hull.append(p)
        while len(hull) > 2 and hull[-2].orientation(hull[-3], hull[-1]) is not Or.LEFT:
            del hull[-2]
    return hull

In [ ]:
print(graham_scan([Point(0,0), Point(1,0.5), Point(2.5,0.25), Point(1,1), Point(2,2), Point(3,0)]))

should handle degeneracies and be somewhat robust

### 3.3 Gift Wrapping
Also called *Jarvis's March*. Output-sensitive.

In [ ]:
def gift_wrapping(points: list[Point]) -> list[Point]:
    current_point = points[0]
    for p in points[1:]:
        if p.x > current_point.x or (p.x == current_point.x and p.y > current_point.y):
            current_point = p
    hull = [current_point]
    while True:
        next_point = points[0]
        for p in filter(lambda p: p != current_point and p != next_point, points[1:]):
            if p.orientation(current_point, next_point) is not Or.RIGHT:
                next_point = p
        if next_point == hull[0]:
            break
        hull.append(next_point)
        current_point = next_point
    return hull

In [ ]:
print(gift_wrapping([Point(0,0), Point(1,0.5), Point(2.5,0.25), Point(2,1), Point(2,2), Point(3,0)]))

### 3.4 Chan's Hull
Best of both worlds

In [ ]:
def tangent_binary_search(mh: list[Point], current_point: Point) -> Point:    # hacky, buggy...
    if len(mh) == 1:
        return mh[0]
    l = 0
    r = len(mh) - 1
    while l <= r:
        m = int((l + r) / 2)
        if (m == 0 or (mh[m-1].orientation(current_point, mh[m]) is Or.RIGHT)) and \
                (m == len(mh) - 1 or (mh[m+1].orientation(current_point, mh[m]) is Or.RIGHT)):
            return mh[m]
        elif (m > 0 and (mh[m-1].orientation(current_point, mh[m]) is Or.RIGHT) != (mh[l].orientation(current_point, mh[r]) is Or.RIGHT)) \
                or (m < len(mh) - 1 and (mh[m+1].orientation(current_point, mh[m]) is Or.RIGHT) != (mh[r].orientation(current_point, mh[l]) is Or.RIGHT)):
            r = m - 1
        else:
            l = m + 1
    return mh[m]

from typing import Optional

def chans_hull_m(points: list[Point], m: int) -> Optional[list[Point]]:
    mini_hulls = []
    for i in range(0, len(points), m):
        j = min(len(points), i+m)
        mini_hulls.append(graham_scan(points[i:j]))
    current_point = points[0]
    for p in points[1:]:
        if p.x > current_point.x or (p.x == current_point.x and p.y > current_point.y):
            current_point = p
    hull = [current_point]
    for _ in range(m-1):           # gift wrapping code zentral auslagern
        candidates = []
        for mh in mini_hulls:
            candidates.append(tangent_binary_search(mh, current_point))
        next_point = candidates[0]
        for p in filter(lambda p: p != current_point and p != next_point, candidates[1:]):
            if p.orientation(current_point, next_point) is not Or.RIGHT:
                next_point = p
        if next_point == hull[0]:
            return hull             # sicherstellen, dass dieser Punkt erreicht wird
        hull.append(next_point)
        current_point = next_point
    return None

def chans_hull(points: list[Point]) -> list[Point]:
    hull = chans_hull_m(points, 2)
    t = 0
    while hull is None:
        t += 1
        m = min(len(points), 2**(2**t))
        hull = chans_hull_m(points, m)
    return hull

In [ ]:
print(chans_hull([Point(0,0), Point(1,0.5), Point(2.5,0.25), Point(2,1), Point(2,2), Point(3,0)]))